In [1]:
import pandas as pd

In [2]:
from pyspark.sql import SparkSession

In [3]:
from pyspark.sql.functions import count, col, when

In [4]:
spark = SparkSession.builder.appName("Funcionarios").getOrCreate()


23/11/29 16:01:25 WARN Utils: Your hostname, evelyn-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/11/29 16:01:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/29 16:01:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
df = spark.read.csv("HR_Analytics.csv", header=True, inferSchema=True)


In [6]:
df.limit(5).toPandas()

23/11/29 16:02:14 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [7]:
df.printSchema()

root
 |-- Age: integer (nullable = true)
 |-- Attrition: string (nullable = true)
 |-- BusinessTravel: string (nullable = true)
 |-- DailyRate: integer (nullable = true)
 |-- Department: string (nullable = true)
 |-- DistanceFromHome: integer (nullable = true)
 |-- Education: integer (nullable = true)
 |-- EducationField: string (nullable = true)
 |-- EmployeeCount: integer (nullable = true)
 |-- EmployeeNumber: integer (nullable = true)
 |-- EnvironmentSatisfaction: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- HourlyRate: integer (nullable = true)
 |-- JobInvolvement: integer (nullable = true)
 |-- JobLevel: integer (nullable = true)
 |-- JobRole: string (nullable = true)
 |-- JobSatisfaction: integer (nullable = true)
 |-- MaritalStatus: string (nullable = true)
 |-- MonthlyIncome: integer (nullable = true)
 |-- MonthlyRate: integer (nullable = true)
 |-- NumCompaniesWorked: integer (nullable = true)
 |-- Over18: string (nullable = true)
 |-- OverTime: string 

In [8]:
novos_nomes = [
    'Idade', 'Atritoo', 'ViagemNegocios', 'TaxaDiaria', 'Departamento', 'DistanciaCasa', 'Educacao',
    'AreaFormacao', 'ContagemFuncionarios', 'NumeroFuncionario', 'SatisfacaoAmbiente', 'Genero',
    'TaxaHora', 'EnvolvimentoTrabalho', 'NivelCargo', 'Cargo', 'SatisfacaoTrabalho', 'EstadoCivil',
    'RendaMensal', 'TaxaMensal', 'NumEmpresasTrabalhadas', 'MaiorIdade', 'HorasExtras', 'AumentoSalarioPerc',
    'AvaliacaoDesempenho', 'SatisfacaoRelacionamento', 'HorasPadrao', 'NivelOpcaoAcoes', 'AnosTrabalhadosTotal',
    'TreinamentoUltimoAno', 'EquilibrioTrabalhoVida', 'AnosEmpresa', 'AnosCargoAtual', 'AnosUltimaPromocao',
    'AnosGerenteAtual'
]

for col_antiga, col_nova in zip(df.columns, novos_nomes):
    df = df.withColumnRenamed(col_antiga, col_nova)


In [9]:
df.printSchema()

root
 |-- Idade: integer (nullable = true)
 |-- Atritoo: string (nullable = true)
 |-- ViagemNegocios: string (nullable = true)
 |-- TaxaDiaria: integer (nullable = true)
 |-- Departamento: string (nullable = true)
 |-- DistanciaCasa: integer (nullable = true)
 |-- Educacao: integer (nullable = true)
 |-- AreaFormacao: string (nullable = true)
 |-- ContagemFuncionarios: integer (nullable = true)
 |-- NumeroFuncionario: integer (nullable = true)
 |-- SatisfacaoAmbiente: integer (nullable = true)
 |-- Genero: string (nullable = true)
 |-- TaxaHora: integer (nullable = true)
 |-- EnvolvimentoTrabalho: integer (nullable = true)
 |-- NivelCargo: integer (nullable = true)
 |-- Cargo: string (nullable = true)
 |-- SatisfacaoTrabalho: integer (nullable = true)
 |-- EstadoCivil: string (nullable = true)
 |-- RendaMensal: integer (nullable = true)
 |-- TaxaMensal: integer (nullable = true)
 |-- NumEmpresasTrabalhadas: integer (nullable = true)
 |-- MaiorIdade: string (nullable = true)
 |-- Horas

In [10]:
contagem_viagem_negocios = df.groupBy('ViagemNegocios').count().show()

+-----------------+-----+
|   ViagemNegocios|count|
+-----------------+-----+
|Travel_Frequently|  277|
|       Non-Travel|  150|
|    Travel_Rarely| 1043|
+-----------------+-----+



In [11]:
contagem_departamento = df.groupBy('Departamento').count().show()

+--------------------+-----+
|        Departamento|count|
+--------------------+-----+
|               Sales|  446|
|Research & Develo...|  961|
|     Human Resources|   63|
+--------------------+-----+



In [12]:
contagem_area_formacao = df.groupBy('AreaFormacao').count().show()

+----------------+-----+
|    AreaFormacao|count|
+----------------+-----+
|Technical Degree|  132|
|           Other|   82|
|       Marketing|  159|
|         Medical|  464|
|   Life Sciences|  606|
| Human Resources|   27|
+----------------+-----+



In [13]:
df = df.withColumn(
    'ViagemNegocios',
    when(df['ViagemNegocios'] == 'Travel_Frequently', 'Frequente').
    when(df['ViagemNegocios'] == 'Non-Travel', 'Sem viagens').
    when(df['ViagemNegocios'] == 'Travel_Rarely', 'Raramente').
    otherwise(df['ViagemNegocios'])
)

In [14]:
contagem_viagem_negocios = df.groupBy('ViagemNegocios').count().show()

+--------------+-----+
|ViagemNegocios|count|
+--------------+-----+
|   Sem viagens|  150|
|     Raramente| 1043|
|     Frequente|  277|
+--------------+-----+



In [15]:
df = df.withColumn(
    'Departamento',
    when(df['Departamento'] == 'Sales', 'Vendas').
    when(df['Departamento'] == 'Research & Development', 'Pesquisa & Desenvolvimento').
    when(df['Departamento'] == 'Human Resources', 'Recursos Humanos').
    otherwise(df['Departamento'])
)
contagem_departamento = df.groupBy('Departamento').count().show()

+--------------------+-----+
|        Departamento|count|
+--------------------+-----+
|    Recursos Humanos|   63|
|Pesquisa & Desenv...|  961|
|              Vendas|  446|
+--------------------+-----+



In [16]:
df = df.withColumn(
    'AreaFormacao',
    when(df['AreaFormacao'] == 'Technical Degree', 'Diploma Técnico').
    when(df['AreaFormacao'] == 'Other', 'Outro').
    when(df['AreaFormacao'] == 'Marketing', 'Marketing').
    when(df['AreaFormacao'] == 'Medical', 'Médica').
    when(df['AreaFormacao'] == 'Life Sciences', 'Ciências da Vida').
    when(df['AreaFormacao'] == 'Human Resources', 'Recursos Humanos').
    otherwise(df['AreaFormacao'])
)
contagem_area_formacao = df.groupBy('AreaFormacao').count().show()

+----------------+-----+
|    AreaFormacao|count|
+----------------+-----+
|Recursos Humanos|   27|
|       Marketing|  159|
|Ciências da Vida|  606|
|          Médica|  464|
| Diploma Técnico|  132|
|           Outro|   82|
+----------------+-----+



In [17]:
df = df.withColumn(
    'Atritoo',
    when(df['Atritoo'] == 'Yes', 'Sim').
    when(df['Atritoo'] == 'No', 'Nao').
    otherwise(df['Atritoo'])
)

In [18]:
df.limit(5).toPandas()

,Idade,Atritoo,ViagemNegocios,TaxaDiaria,Departamento,DistanciaCasa,Educacao,AreaFormacao,ContagemFuncionarios,NumeroFuncionario,...,SatisfacaoRelacionamento,HorasPadrao,NivelOpcaoAcoes,AnosTrabalhadosTotal,TreinamentoUltimoAno,EquilibrioTrabalhoVida,AnosEmpresa,AnosCargoAtual,AnosUltimaPromocao,AnosGerenteAtual
0,41,Sim,Raramente,1102,Vendas,1,2,Ciências da Vida,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,Nao,Frequente,279,Pesquisa & Desenvolvimento,8,1,Ciências da Vida,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Sim,Raramente,1373,Pesquisa & Desenvolvimento,2,2,Outro,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,Nao,Frequente,1392,Pesquisa & Desenvolvimento,3,4,Ciências da Vida,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,Nao,Raramente,591,Pesquisa & Desenvolvimento,2,1,Médica,1,7,...,4,80,1,6,3,3,2,2,2,2


In [19]:
contagem_educacao = df.groupBy('Educacao').count().show()

+--------+-----+
|Educacao|count|
+--------+-----+
|       1|  170|
|       3|  572|
|       5|   48|
|       4|  398|
|       2|  282|
+--------+-----+



In [20]:
df = df.withColumn(
    'Educacao',
    when(df['Educacao'] == 1, 'Ensino Médio').
    when(df['Educacao'] == 2, 'Graduação').
    when(df['Educacao'] == 3, 'Bacharelado').
    when(df['Educacao'] == 4, 'Mestrado').
    when(df['Educacao'] == 5, 'Doutorado').
    otherwise(df['Educacao'])
)

In [21]:
contagem_educacao = df.groupBy('Educacao').count().show()

+------------+-----+
|    Educacao|count|
+------------+-----+
|   Doutorado|   48|
|Ensino Médio|  170|
| Bacharelado|  572|
|    Mestrado|  398|
|   Graduação|  282|
+------------+-----+



In [22]:
contagem_genero = df.groupBy('Genero').count().show()

+------+-----+
|Genero|count|
+------+-----+
|Female|  588|
|  Male|  882|
+------+-----+



In [24]:
df = df.withColumn(
    'Genero',
    when(df['Genero'] == 'Female', 'Mulher').
    when(df['Genero'] == 'Male', 'Homem').
    otherwise(df['Genero'])
)

In [26]:
contagem_genero = df.groupBy('Genero').count().show()

+------+-----+
|Genero|count|
+------+-----+
| Homem|  882|
|Mulher|  588|
+------+-----+



In [29]:
contagem_cargo = df.groupBy('Cargo').count()
contagem_cargo.toPandas()

,Cargo,count
0,Sales Executive,326
1,Manufacturing Director,145
2,Laboratory Technician,259
3,Sales Representative,83
4,Healthcare Representative,131
5,Research Scientist,292
6,Manager,102
7,Research Director,80
8,Human Resources,52


In [30]:
df = df.withColumn(
    'Cargo',
    when(df['Cargo'] == 'Sales Executive', 'Executivo de Vendas').
    when(df['Cargo'] == 'Manufacturing Director', 'Diretor de Manufatura').
    when(df['Cargo'] == 'Laboratory Technician', 'Técnico de Laboratório').
    when(df['Cargo'] == 'Sales Representative', 'Representante de Vendas').
    when(df['Cargo'] == 'Healthcare Representative', 'Representante de Saúde').
    when(df['Cargo'] == 'Research Scientist', 'Cientista de Pesquisa').
    when(df['Cargo'] == 'Manager', 'Gerente').
    when(df['Cargo'] == 'Research Director', 'Diretor de Pesquisa').
    when(df['Cargo'] == 'Human Resources', 'Recursos Humanos').
    otherwise(df['Cargo'])
)

In [31]:
contagem_cargo = df.groupBy('Cargo').count()
contagem_cargo.toPandas()

,Cargo,count
0,Recursos Humanos,52
1,Gerente,102
2,Executivo de Vendas,326
3,Cientista de Pesquisa,292
4,Diretor de Pesquisa,80
5,Técnico de Laboratório,259
6,Diretor de Manufatura,145
7,Representante de Saúde,131
8,Representante de Vendas,83


In [32]:
contagem_estadocivil= df.groupBy('EstadoCivil').count()
contagem_estadocivil.toPandas()

,EstadoCivil,count
0,Married,673
1,Divorced,327
2,Single,470


In [33]:
df = df.withColumn(
    'EstadoCivil',
    when(df['EstadoCivil'] == 'Married', 'Casado(a)').
    when(df['EstadoCivil'] == 'Divorced', 'Divorciado(a)').
    when(df['EstadoCivil'] == 'Single', 'Solteiro(a)').
    otherwise(df['EstadoCivil'])
)

In [34]:
contagem_estadocivil= df.groupBy('EstadoCivil').count()
contagem_estadocivil.toPandas()

,EstadoCivil,count
0,Divorciado(a),327
1,Casado(a),673
2,Solteiro(a),470


In [35]:
contagem_maioridade= df.groupBy('Maioridade').count()
contagem_maioridade.toPandas()

,Maioridade,count
0,Y,1470


In [36]:
contagem_Horasextras= df.groupBy('horasExtras').count()
contagem_Horasextras.toPandas()

,horasExtras,count
0,No,1054
1,Yes,416


In [37]:
df = df.withColumn(
    'horasExtras',
    when(df['horasExtras'] == 'Yes', 'Sim').
    when(df['horasExtras'] == 'No', 'Não').
    otherwise(df['horasExtras'])
)

In [38]:
contagem_Horasextras= df.groupBy('horasExtras').count()
contagem_Horasextras.toPandas()

,horasExtras,count
0,Não,1054
1,Sim,416


In [39]:
df_atrito = df.filter(col('Atritoo') == 'Sim')

df_atrito.limit(5).toPandas()

,Idade,Atritoo,ViagemNegocios,TaxaDiaria,Departamento,DistanciaCasa,Educacao,AreaFormacao,ContagemFuncionarios,NumeroFuncionario,...,SatisfacaoRelacionamento,HorasPadrao,NivelOpcaoAcoes,AnosTrabalhadosTotal,TreinamentoUltimoAno,EquilibrioTrabalhoVida,AnosEmpresa,AnosCargoAtual,AnosUltimaPromocao,AnosGerenteAtual
0,41,Sim,Raramente,1102,Vendas,1,Graduação,Ciências da Vida,1,1,...,1,80,0,8,0,1,6,4,0,5
1,37,Sim,Raramente,1373,Pesquisa & Desenvolvimento,2,Graduação,Outro,1,4,...,2,80,0,7,3,3,0,0,0,0
2,28,Sim,Raramente,103,Pesquisa & Desenvolvimento,24,Bacharelado,Ciências da Vida,1,19,...,2,80,0,6,4,3,4,2,0,3
3,36,Sim,Raramente,1218,Vendas,9,Mestrado,Ciências da Vida,1,27,...,2,80,0,10,4,3,5,3,0,3
4,34,Sim,Raramente,699,Pesquisa & Desenvolvimento,6,Ensino Médio,Médica,1,31,...,3,80,0,8,2,3,4,2,1,3


In [40]:
dados_atrito2 = df_atrito.coalesce(1)

dados_atrito2.write.csv('df_atrito2.csv', header=True)

In [41]:
dados_atrito = df.coalesce(1)

dados_atrito.write.csv('df_atrito.csv', header=True)